## Classic Coroutines

Classic coroutines in Python refer to an older style of coroutine implementation that was available prior to the introduction of native coroutines in Python 3.5. Classic coroutines were implemented using generator functions and the `yield` keyword.

In the context of Python, coroutines are a way to write functions that can be suspended and resumed later while maintaining their internal state. This allows for cooperative multitasking, where different coroutines can take turns executing and potentially interleave their operations.

To create a classic coroutine, you define a generator function using the `def` keyword, and within the function, you use the `yield` keyword to suspend execution and produce a value. The generator function becomes a coroutine by calling it and obtaining a generator object.

Here's an example of a classic coroutine that counts from 1 to 3:

```python
def counter():
    i = 1
    while True:
        yield i
        i += 1

# Create a coroutine
coro = counter()

# Print the values produced by the coroutine
print(next(coro))  # 1
print(next(coro))  # 2
print(next(coro))  # 3
```

In this example, the `counter` function is a classic coroutine because it uses the `yield` keyword. When `coro` is created by calling `counter()`, it returns a generator object. By calling `next(coro)`, we can advance the coroutine's execution and obtain the values it produces.

Classic coroutines can also receive values from the caller using the `send` method. By doing so, the value sent becomes the result of the `yield` expression inside the coroutine. Here's an example:

```python
def printer():
    while True:
        value = yield
        print(value)

# Create a coroutine
coro = printer()

# Start the coroutine
next(coro)

# Send values to the coroutine
coro.send("Hello")
coro.send("World")
```

In this example, the `printer` function is a classic coroutine that receives values using the `yield` expression. We start the coroutine by calling `next(coro)` to advance to the first `yield` statement. Then, we can send values to the coroutine using the `send` method, and these values are printed by the coroutine.

Classic coroutines were a precursor to native coroutines, which were introduced in Python 3.5 with the `async` and `await` keywords. Native coroutines provide better syntax and improved performance compared to classic coroutines, and they are the recommended approach for writing asynchronous code in modern versions of Python.

In [1]:
def printer():
    while True:
        value = yield
        print(value)

coro = printer()
next(coro)
coro.send("Hello")
coro.send("World")

Hello
World


### Basic Behavior of a Coroutine

Classic coroutines are based on generators and use the `yield` statement for pausing and resuming execution.

Let's consider an example to understand the basic behavior of a classic-style coroutine that receives and sends values. Suppose we want to implement a coroutine that calculates the running average of a sequence of numbers. Here's how it might look:

```python
def average_coroutine():
    count = 0
    total = 0
    average = None

    while True:
        value = yield average
        total += value
        count += 1
        average = total / count
```

In this example, we define the `average_coroutine()` function as a classic-style coroutine. It initializes variables for count, total, and average. The coroutine enters an infinite loop using `while True`. Inside the loop, it uses the `yield` statement to pause execution and yield the current average value.

To send values to the coroutine and receive the updated average, we can use the `send()` method. Here's an example usage:

```python
coroutine = average_coroutine()
next(coroutine)  # Prime the coroutine

values = [10, 20, 30, 40, 50]
for value in values:
    average = coroutine.send(value)
    print(f"Received average: {average}")
```

In this example, we first create an instance of the `average_coroutine()` coroutine by calling it. We then prime the coroutine by calling `next(coroutine)` to advance it to the first `yield` statement.

Next, we have a list of values [10, 20, 30, 40, 50]. We iterate over these values and send each value to the coroutine using `coroutine.send(value)`. This sends the value to the coroutine and resumes its execution until it reaches the next `yield` statement. The coroutine updates the count, total, and average variables and yields the current average value.

In the loop, we print the received average after each value is sent to the coroutine. The output will show the running average as the coroutine processes each value in the sequence.

In [2]:
def average_coroutine():
    count = 0
    total = 0
    average = None

    while True:
        value = yield average
        total += value
        count += 1
        average = total / count

In [4]:
avg = average_coroutine()
next(avg)

In [5]:
avg.send(10)

10.0

In [6]:
avg.send(20)

15.0

### States of coroutine

A coroutine can exist in four different states. The current state of a coroutine can be determined using the `inspect.getgeneratorstate(...)` function, which returns one of the following strings:

1. `'GEN_CREATED'`:
   This state indicates that the coroutine is waiting to start its execution.

2. `'GEN_RUNNING'`:
   This state indicates that the coroutine is currently being executed by the interpreter.

3. `'GEN_SUSPENDED'`:
   This state indicates that the coroutine is currently suspended at a `yield` expression.

4. `'GEN_CLOSED'`:
   This state indicates that the execution of the coroutine has been completed.

When using the `send` method, the argument provided will become the value of the pending `yield` expression. Therefore, you can only call something like `my_coro.send(42)` if the coroutine is currently suspended. However, this is not the case if the coroutine has never been activated, i.e., when its state is 'GEN_CREATED'. That's why the first activation of a coroutine is always done using `next(my_coro)`. Alternatively, you can also call `my_coro.send(None)`, and the effect will be the same.

In [1]:
import inspect

def coroutine():
    while True:
        x = yield
        print(x)

In [2]:
c = coroutine()

inspect.getgeneratorstate(c)

'GEN_CREATED'

In [3]:
next(c)
inspect.getgeneratorstate(c)

'GEN_SUSPENDED'

> **<span style="color:red">Note:<span/>** If you try to send a value to a coroutine without first calling `next` on it or calling `.send(None)` you will get an error.

In [4]:
c = coroutine()
c.send(10)

TypeError: can't send non-None value to a just-started generator

> **Let's now look at a coroutine in closed state:**

In [9]:
import inspect

def coroutine():
    x = yield
    print(x)
    y = yield
    print(y)

In [10]:
c = coroutine()

In [11]:
next(c)

In [12]:
c.send('Hello')

Hello


In [13]:
c.send('!!!')

!!!


StopIteration: 

In [14]:
inspect.getgeneratorstate(c)

'GEN_CLOSED'

It is of utmost importance to grasp the concept that the execution of the coroutine pauses precisely at the yield keyword. The code on the right side of the equals sign (`=`) is evaluated before the assignment itself takes place. Consequently, in a line such as `b = yield a`, the value of `b` will only be assigned when the coroutine is subsequently activated by the client code. Adjusting to this reality may require some effort, but comprehending it is crucial for comprehending the application of yield in asynchronous programming, as we will explore later on.

> **Let's now break down the execution of a coroutine which computes the running average:**

In [16]:
def average_coroutine():
    print('Starting coroutine...')
    count = 0
    total = 0
    average = None

    while True:
        value = yield average
        if value is None:
            break
        total += value
        count += 1
        average = total / count

The execution of the given coroutine can be broken down into three stages: coroutine priming, sending a value to the coroutine, and stopping the coroutine by sending None.

In [18]:
averager = average_coroutine()

In [19]:
next(averager)

Starting coroutine...


1. Coroutine Priming:
   - When the coroutine is initially created and called, it doesn't start executing immediately. Instead, it enters the priming stage.
   - At this stage, the coroutine's code is not yet executed. The initial execution is paused at the first line of the coroutine function, which is `print('Starting coroutine...')`.
   - The coroutine is now ready to receive a value through the `yield` expression.

In [20]:
averager.send(10)

10.0

In [21]:
averager.send(3)

6.5

2. Sending a Value to the Coroutine:
   - To resume the execution of the coroutine and proceed beyond the `yield` expression, a value needs to be sent to it.
   - The value is sent using the `send()` method, which can be called on the coroutine object.
   - When a value is sent, the execution of the coroutine continues from where it left off, and the value is assigned to the `value` variable in the coroutine.
   - In the given coroutine, the `yield` expression is `yield average`, so the value sent to the coroutine will be assigned to the `average` variable.
   - The execution of the coroutine proceeds until the next `yield` expression is encountered.

In [22]:
averager.send(None)

StopIteration: 

3. Stopping the Coroutine by Sending `None`:
   - To stop the execution of the coroutine, a `None` value is sent through the `send()` method.
   - In the given coroutine, when the value received is None, the `if value is None` condition is satisfied, and the loop is terminated using the `break` statement.
   - After the loop is exited, the coroutine function ends, and the coroutine is considered complete.

### Priming a Coroutine Using Decorator

Priming a coroutine refers to the process of preparing a coroutine to be executed by advancing it to its first `yield` statement. In Python, a common way to prime a coroutine is by using a decorator.

Here's an example of how you can prime a coroutine using a decorator in Python:

```python
def coroutine_decorator(func):
    def wrapper(*args, **kwargs):
        coroutine = func(*args, **kwargs)
        next(coroutine)  # Advance the coroutine to the first yield statement
        return coroutine
    return wrapper

@coroutine_decorator
def my_coroutine():
    while True:
        value = yield
        # Process the value or perform some other task

# Create an instance of the primed coroutine
coro = my_coroutine()

# Send values to the coroutine
coro.send('Hello')
coro.send('World')
```

In the code above, we define a decorator function `coroutine_decorator` that takes a coroutine function as an argument. Inside the decorator, we create a wrapper function that calls the coroutine function, advances it to the first `yield` statement using `next(coroutine)`, and returns the primed coroutine.

The `my_coroutine` function is decorated with `@coroutine_decorator`, which means that when we create an instance of `my_coroutine` using `coro = my_coroutine()`, it will be automatically primed.

Once the coroutine is primed, we can send values to it using the `send()` method. In the example, we send the strings `'Hello'` and `'World'` to the coroutine using `coro.send('Hello')` and `coro.send('World')`. The coroutine can process these values or perform any other task defined within its body.

Note that after the first `yield` statement, subsequent calls to `send()` will resume the coroutine from where it left off, allowing it to receive and process values.

In [27]:
def coroutine_decorator(func):
    def wrapper(*args, **kwargs):
        coroutine = func(*args, **kwargs)
        next(coroutine)  # Advance the coroutine to the first yield statement
        return coroutine
    return wrapper


@coroutine_decorator
def average_coroutine():
    print('Starting coroutine...')
    count = 0
    total = 0
    average = None

    while True:
        value = yield average
        if value is None:
            break
        total += value
        count += 1
        average = total / count

In [24]:
averager = average_coroutine()

Starting coroutine...


In [25]:
averager.send(1)

1.0

In [26]:
averager.send(11)

6.0

### Throwing Exceptions to Coroutines

The `throw` method allows you to raise an exception from outside the coroutine's code, effectively interrupting its execution at the next `yield` point. Here's how it works:

1. **Exception Injection:** You call `throw` on the coroutine object, passing the exception type (`type`) and optionally the exception value (`value`) and traceback (`traceback`).
2. **Suspension Point:** If the coroutine is currently suspended at a `yield` expression, the exception is raised as if it originated from within the coroutine at that point.
3. **Exception Handling:** The coroutine's code can handle the exception using a `try...except` block around the `yield` expression. If handled, execution resumes from the next statement after the `yield`.
4. **Unhandled Exception:** If the exception is not handled within the coroutine, it propagates back to the caller (the code that called `throw`).

Here's an example of how it works:

```python
def coroutine():
    while True:
        try:
            x = yield
            print(x)
        except ZeroDivisionError:
            print('Please do not divide by Zero!')
```

Let's break down what it does:

1. The coroutine is defined using the `def` keyword and named `coroutine()`.
2. Within the coroutine, there is an infinite loop indicated by `while True`. This means the coroutine will keep executing its code indefinitely, unless it is explicitly stopped or interrupted.
3. Inside the loop, there is a `try-except` block that handles exceptions.
4. The line `x = yield` is a generator statement that acts as a pause point within the coroutine. It allows the coroutine to receive a value from the code that is consuming the coroutine.
   - When the coroutine is first started, calling `next(coroutine)` or `coroutine.send(None)` will advance the coroutine to this point, and the `yield` expression will evaluate to `None`.
   - When a value is sent to the coroutine using `coroutine.send(value)`, the coroutine resumes execution from the `yield` expression, and the value sent is assigned to the variable `x`.
   - The coroutine then proceeds to print the value of `x`.
5. If a `ZeroDivisionError` exception is raised within the coroutine by sending it through `.throw` method, the `except ZeroDivisionError` block is executed.
   - In this case, the coroutine prints the message "Please do not divide by Zero!".


In conclusion `throw` lets you forcefully insert an error at that pause point. If the generator has a way to deal with the error (like a try...except block), it can handle it and continue and the next value yielded becomes the value of the `c.throw(Exception)` . Otherwise, the error gets thrown back to wherever you called the generator from.

In [49]:
def coroutine():
    while True:
        try:
            x = yield
            print(x)
        except ZeroDivisionError:
            print('Please do not divide by Zero!')

In [50]:
c = coroutine()

In [51]:
next(c)

In [52]:
c.send(10)

10


In [53]:
c.send(100)

100


In [54]:
c.throw(ZeroDivisionError)

Please do not divide by Zero!


In [55]:
c.send(10)

10


> **<span style="color:red">Note:<span/>** If the exception is not handled within the coroutine, it propagates back to the caller.

In [56]:
c.throw(Exception)

Exception: 

### Terminating a Coroutine

The `close` method on classic-style coroutines serves to gracefully terminate the coroutine's execution and potentially perform cleanup tasks. Here's a breakdown of its functionality:

**Functionality:**

- **Signals Termination:** When you call `close` on a coroutine object, it signals the coroutine to stop execution at the next convenient point. This "convenient point" is typically the next `yield` expression.
- **Cleanup Actions:** The `close` method can also be used to trigger any necessary cleanup actions within the coroutine, such as closing open files or releasing resources. This is often achieved using the `finally` block of a `try...except` construct surrounding `yield` expressions.
- **Exception Generation:** Internally, `close` raises a special exception called `GeneratorExit`. This exception can be caught within the coroutine's code using a `try...except` block.

Here's an example of how it works:

```python
def my_coroutine():
    try:
        while True:
            resource = open("data.txt", "w")
            data = yield  # Suspend at the first yield
            resource.write(data)
            print(data)
    except GeneratorExit:  # Catch the special exception raised by close
        print("Coroutine terminated by close")
    finally:
        print("Cleanning up...")
        resource.close()

coro = my_coroutine()

# Simulate successful execution
value = next(coro)
coro.send('Hello')

# Terminate the coroutine gracefully
coro.close()

# Output:
# 5
# Coroutine terminated by close
# Cleanning up...
```

In this example:

- The coroutine opens a file (`data.txt`) and suspends at the first `yield`.
- We send a value (`'Hello'`) at the first `yield`.
- Calling `close` on the coroutine object raises `GeneratorExit`, which is caught by the `try...except` block.
- The `finally` block ensures the file is closed.
- The coroutine execution is terminated, and a message indicating termination is printed.

In [103]:
def my_coroutine():
    try:
        while True:
            resource = open("data.txt", "w")
            data = yield  # Suspend at the first yield
            resource.write(data)
            print(data)
    except GeneratorExit:  # Catch the special exception raised by close
        print("Coroutine terminated by close")
    finally:
        print("Cleanning up...")
        resource.close()

In [104]:
c = my_coroutine()
next(c)

In [105]:
c.send('Hello')
c.close()

Hello
Coroutine terminated by close
Cleanning up...


> **<span style="color:red">Note:</span>** When a coroutine is about to be garbage collected, the `.close()` method on it will be called before it gets garbage collected.

In [18]:
def coroutine():
    try:
        yield 'Hello'
        yield 'World'
        return 'Done'
    except GeneratorExit:
        print('Exiting...')

In [19]:
c = coroutine()

In [20]:
next(c)

'Hello'

In [21]:
del c

Exiting...


### Return Statement Inside a Coroutine

In classic-style coroutines, a `return` statement within the coroutine function behaves differently compared to regular functions. Here's what it does:

**Behavior:**

- **Terminates Execution:** Unlike a regular function where `return` exits the function and returns a value, in a coroutine, `return` permanently terminates the coroutine's execution.
- **No Return Value:** Classic coroutines themselves don't have a formal return value. However, the value you provide to the `return` statement can be used in specific contexts.

- **Return Value is Inside an `StopIteration` Exception**: The returned value can be extracted by handling the raised exception and obtaining the value of exception.

**Example Usage:**

```python
from collections import namedtuple

Result = namedtuple('Result', 'count average')


def average_coroutine():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break  # (1)
        total += term
        count += 1
        average = total/count
    return Result(count, average)

averager = average_coroutine()
next(averager)

averager.send(10)
averager.send(2)

try:
    averager.send(None)
except StopIteration as e:
    print(e.value)
```

This code defines a coroutine `average_coroutine()` that calculates the average of numbers sent to it using the `send()` method. Let's break down the code step by step:

1. **Importing necessary modules**: The code imports the `namedtuple` class from the `collections` module. `namedtuple` is used to create a simple class to store the result of the average calculation.

2. **Defining the `Result` named tuple**: A named tuple called `Result` is created with two fields: `count` and `average`. This tuple will be used to store the result of the average calculation.

3. **Defining the `average_coroutine()` function**: This function is a generator-based coroutine. It initializes three variables: `total` (sum of numbers), `count` (number of numbers), and `average` (current average). It enters an infinite loop (`while True`) and waits for input using the `yield` keyword.

4. **Receiving input**: Inside the loop, the coroutine receives input using `term = yield`. It expects numbers to be sent to it via `send()`. If `None` is sent (signaling the end of input), the loop breaks (marked as (1)).

5. **Calculating the average**: If a number is received (`term is not None`), it adds the number to `total`, increments `count`, and updates the `average` accordingly.

6. **Returning the result**: When `None` is received, indicating the end of input, the coroutine exits the loop, creates a `Result` named tuple with the final `count` and `average`, and returns it.

7. **Instantiating the coroutine and preparing for input**: The `average_coroutine()` is instantiated and initialized by calling `next(averager)`, which moves the coroutine to the first `yield` statement.

8. **Sending numbers to the coroutine**: Numbers (10 and 2) are sent to the coroutine using `send()`. The coroutine calculates the average as numbers are sent.

9. **Ending input and retrieving the result**: Finally, `None` is sent to the coroutine to signal the end of input. The coroutine calculates the final average and raises a `StopIteration` exception with the result. The result is then accessed using `e.value` and printed.

**Important Notes:**

- `return` in classic coroutines is primarily used for termination, not for returning values like regular functions.

In [108]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def average_coroutine():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break  # (1)
        total += term
        count += 1
        average = total/count
    return Result(count, average)

averager = average_coroutine()
next(averager)

averager.send(10)
averager.send(2)

try:
    averager.send(None)
except StopIteration as e:
    print(e.value)

Result(count=2, average=6.0)


### Extra curriculum:
1. [Classic Coroutines](https://www.fluentpython.com/extra/classic-coroutines)
2. [Curious Course on Coroutines and Concurrency](https://www.youtube.com/watch?v=Z_OAlIhXziw)